# Automated CapsnetsS2I train + test

First, import stuff

In [1]:
import model_s2i
import data_loader
import tensorflow as tf

W0823 14:10:09.274354 4527914432 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Load word vectors and data

In [3]:
WORD_VEC_SCENARIO = "FastText-300-verbsDatasets-synAntVSP-dia"
SCENARIO_NAME = 'rerouting-3-attention'

word2vec_path = '../../romanian_word_vecs/cleaned-vectors-diacritice.vec'

# No diacritics data
training_data_paths = [
#         '../data-capsnets/scenario0/train.txt',
#         '../data-capsnets/scenario1/train.txt',
#         '../data-capsnets/scenario2/train.txt',
#         '../data-capsnets/scenario3.1/train.txt',
#         '../data-capsnets/scenario3.2/train.txt',
#         '../data-capsnets/scenario3.3/train.txt',
        '../data-capsnets/diacritics/scenario0/train.txt',
        '../data-capsnets/diacritics/scenario1/train.txt',
        '../data-capsnets/diacritics/scenario2/train.txt',
        '../data-capsnets/diacritics/scenario31/train.txt',
        '../data-capsnets/diacritics/scenario32/train.txt',
        '../data-capsnets/diacritics/scenario33/train.txt',
        
    ]
test_data_paths = [
#     '../data-capsnets/scenario0/test.txt',
#     '../data-capsnets/scenario1/test.txt',
#     '../data-capsnets/scenario2/test.txt',
#     '../data-capsnets/scenario3.1/test.txt',
#     '../data-capsnets/scenario3.2/test.txt',
#     '../data-capsnets/scenario3.3/test.txt',
    '../data-capsnets/diacritics/scenario0/test.txt',
    '../data-capsnets/diacritics/scenario1/test.txt',
    '../data-capsnets/diacritics/scenario2/test.txt',
    '../data-capsnets/diacritics/scenario31/test.txt',
    '../data-capsnets/diacritics/scenario32/test.txt',
    '../data-capsnets/diacritics/scenario33/test.txt',
]

scenario_nums = [
#     '0_' + WORD_VEC_SCENARIO,
#     '1_' + WORD_VEC_SCENARIO,
#     '2_' + WORD_VEC_SCENARIO,
#     '31_' + WORD_VEC_SCENARIO,
#     '32_' + WORD_VEC_SCENARIO,
#     '33_' + WORD_VEC_SCENARIO,
#     '0_dia_' + WORD_VEC_SCENARIO,
#     '1_dia_' + WORD_VEC_SCENARIO,
#     '2_dia_' + WORD_VEC_SCENARIO,
#     '31_dia_' + WORD_VEC_SCENARIO,
#     '32_dia_' + WORD_VEC_SCENARIO,
#     '33_dia_' + WORD_VEC_SCENARIO,
    '0-rerouting-3-attention',
    '1-rerouting-3-attention',
    '2-rerouting-3-attention',
    '31-rerouting-3-attention',
    '32-rerouting-3-attention',
    '33-rerouting-3-attention',
]

In [4]:
print('------------------load word2vec begin-------------------')
w2v = data_loader.load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
loading time took 000.06
------------------load word2vec end---------------------


# Flag setting functions + utils

In [5]:
import csv

def set_flags(data, scenario):
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
    tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
    tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
    tf.app.flags.DEFINE_string('scenario_num', scenario, 'Scenario number')
    tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
    tf.app.flags.DEFINE_string('results_dir', './results/', 'Results dir')
    tf.app.flags.DEFINE_string('hyperparams_dir', './hyperparams/', 'Hyperparams dir')
    
    vocab_size, word_emb_size = data['embedding'].shape
    _, max_sentence_length = data['x_tr'].shape
    intents_number = len(data['intents_dict'])
    slots_number = len(data['slots_dict'])
    hidden_size = 64

    tf.app.flags.DEFINE_float('keep_prob', 1, 'embedding dropout keep rate for training')
    tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
    tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
    tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
    tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
    tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
    tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
    tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
    tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')
    tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
    tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
    tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
    tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
    tf.app.flags.DEFINE_integer('intent_output_dim', 16, 'intent output dimension')
    tf.app.flags.DEFINE_integer('slot_output_dim', 2 * hidden_size, 'slot output dimension')
    tf.app.flags.DEFINE_integer('d_a', 20, 'self attention weight hidden units number')
    tf.app.flags.DEFINE_integer('r', 3, 'number of self attention heads')
    tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
    tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
    tf.app.flags.DEFINE_float('rerouting_coef', 0.5, 'coefficient for rerouting')
    tf.app.flags.DEFINE_boolean('use_rerouting', True, 'whether to use rerouting or not')
    
    tf.app.flags.DEFINE_string('f', '', 'kernel') # Without this there's an error: unknown command line flag 'f'
    
    return FLAGS

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
def write_csv(run_results):
    with open('results/results_{}.csv'.format(SCENARIO_NAME), 'w') as f:
        header_line = ['Scenario', 'Intent F1', 'Slot F1']
        writer = csv.writer(f)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            intent_score = [v['intent_f1']]
            slot_score = [v['slot_f1']]
            l = scenario_num + intent_score + slot_score
            writer.writerow(l)

In [ ]:
# In case it's needed
del_all_flags(FLAGS)

In [6]:
import train
import test

results = dict()
for i in range(len(training_data_paths)):
    
    # Load data
    data = data_loader.read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True)
    
    FLAGS = set_flags(data, scenario=scenario_nums[i])
    
    # Train model
    print('TRAINING ' + scenario_nums[i])
    train.train(model_s2i.CapsNetS2I, data, FLAGS, batches_rand=False)
    
    # Test model
    print('TESTING ' + scenario_nums[i])
    intent_f, slot_f = test.test(model_s2i.CapsNetS2I, data, FLAGS)
    results[scenario_nums[i]] = dict()
    results[scenario_nums[i]]['intent_f1'] = intent_f
    results[scenario_nums[i]]['slot_f1'] = slot_f
        
    # Reset flags
    del_all_flags(FLAGS)

write_csv(results)

------------------read datasets begin-------------------
max length is 15
max length is 15
------------------read datasets end---------------------
TRAINING 0-rerouting-3-attention
------------------start cross-validation-------------------
FOLD 1
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.052381
F score 0.052381
Slot filling
F1 score: 0.033729
Accuracy: 0.384127
------------------epoch :  0  Loss:  19.43955 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.452381
F score 0.452381
Slot filling
F1 score: 0.377914
Accuracy: 0.927619
Current F score mean 0.4151475314052001
Best F score mean 0.4151475314052001
------------------epoch :  1  Loss:  5.3063526 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.661905
F score 0.661905
Slot filli

------------------epoch :  5  Loss:  0.07500332 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.976190
F score 0.976190
Slot filling
F1 score: 0.984802
Accuracy: 0.998889
Current F score mean 0.9804964539007092
Best F score mean 0.9885714285714285
------------------epoch :  6  Loss:  0.005878382 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.969048
F score 0.969048
Slot filling
F1 score: 0.996948
Accuracy: 0.999524
Current F score mean 0.9829978685268614
Best F score mean 0.9885714285714285
------------------epoch :  7  Loss:  0.005083346 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.976190
F score 0.976190
Slot filling
F1 score: 0.990863
Accuracy: 0.999365
Current F score mean 0.9835267101764564
Best F score mean 0.9885714285714285
------------------epoch :  8  Loss:  0.0038743275 ----------------------
           VA

------------------epoch :  11  Loss:  0.002903159 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.995238
F score 0.995238
Slot filling
F1 score: 0.985859
Accuracy: 0.998889
Current F score mean 0.9905483405483405
Best F score mean 0.9977911938706236
------------------epoch :  12  Loss:  0.0026918724 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.995238
F score 0.995238
Slot filling
F1 score: 0.985859
Accuracy: 0.998889
Current F score mean 0.9905483405483405
Best F score mean 0.9977911938706236
------------------epoch :  13  Loss:  0.002523219 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.995238
F score 0.995238
Slot filling
F1 score: 0.985859
Accuracy: 0.998889
Current F score mean 0.9905483405483405
Best F score mean 0.9977911938706236
------------------epoch :  14  Loss:  0.0023839325 ----------------------
       

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

...

------------------epoch :  11  Loss:  0.020062994 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.979798
F score 0.979798
Slot filling
F1 score: 0.984894
Accuracy: 0.996753
Current F score mean 0.9823461198083554
Best F score mean 0.987145207733443
------------------epoch :  12  Loss:  0.01650807 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.979798
F score 0.979798
Slot filling
F1 score: 0.984894
Accuracy: 0.996753
Current F score mean 0.9823461198083554
Best F score mean 0.987145207733443
------------------epoch :  13  Loss:  0.014664769 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.979798
F score 0.979798
Slot filling
F1 score: 0.984894
Accuracy: 0.996753
Current F score mean 0.9823461198083554
Best F score mean 0.987145207733443
------------------epoch :  14  Loss:  0.013288912 ----------------------
           VA

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

Intent F1: 0.262970
Slot F1: 0.452438
